In [2]:
import sklearn
import pandas as pd
import numpy as np
from sklearn import preprocessing


In [3]:
path = "/content/drive/MyDrive/Thesis/Datasets/Turbofan_Dataset/"

In [4]:
#
# TRAINING DATA
#

In [5]:
#Load training data
train_df = pd.read_csv( path + 'Initial_Data/train_FD001.txt', sep=" ", header=None)
train_df.drop(train_df.columns[[26, 27]], axis=1, inplace=True)
train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
# train_df.head()

In [6]:
#Load test data
test_df = pd.read_csv( path + 'Initial_Data/test_FD001.txt', sep=" ", header=None)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
# test_df.head()

In [7]:
#Load ground truth data
truth_df = pd.read_csv( path + 'Initial_Data/RUL_FD001.txt', sep=" ", header=None)
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)
# truth_df.head()

In [9]:
#Make sure timeseries is sorted in time
train_df = train_df.sort_values(['id','cycle'])
# train_df.head()

In [10]:
# Data Labeling - generate column RUL
rul = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
train_df = train_df.merge(rul, on=['id'], how='left')
train_df['RUL'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)
# train_df.head()

In [11]:
train_df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735,4
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594,3
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333,2
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640,1


In [12]:
# 

# Total 100 engines 20630 cycles

# 

In [13]:
# generate label columns for training data
w1 = 30
# w0 = 15
train_df['fail_30'] = np.where(train_df['RUL'] <= w1, 1, 0 )
# train_df['fail_15'] = train_df['fail_30']
# train_df.loc[train_df['RUL'] <= w0, 'fail_15'] = 2
# train_df

In [ ]:
train_df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL,fail_30
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191,0
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190,0
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189,0
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188,0
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,47.28,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,21.61,551.43,2388.19,9065.52,1.3,48.07,519.49,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735,4,1
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,21.61,550.86,2388.23,9065.11,1.3,48.04,519.68,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594,3,1
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,21.61,550.94,2388.24,9065.90,1.3,48.09,520.01,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333,2,1
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,21.61,550.68,2388.25,9073.72,1.3,48.39,519.67,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640,1,1


In [ ]:
# MinMax normalization
train_df['cycle_norm'] = train_df['cycle']
cols_normalize = train_df.columns.difference(['id','cycle','RUL','fail_30'])
min_max_scaler = preprocessing.MinMaxScaler()
norm_train_df = pd.DataFrame(min_max_scaler.fit_transform(train_df[cols_normalize]), 
                             columns=cols_normalize, 
                             index=train_df.index)
join_df = train_df[train_df.columns.difference(cols_normalize)].join(norm_train_df)
train_df = join_df.reindex(columns = train_df.columns)
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL,fail_30,cycle_norm
0,1,1,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,1.0,0.726248,0.242424,0.109755,0.0,0.369048,0.633262,0.205882,0.199608,0.363986,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,0,0.00000
1,1,2,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,1.0,0.628019,0.212121,0.100242,0.0,0.380952,0.765458,0.279412,0.162813,0.411312,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,0,0.00277
2,1,3,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,1.0,0.710145,0.272727,0.140043,0.0,0.250000,0.795309,0.220588,0.171793,0.357445,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,0,0.00554
3,1,4,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,1.0,0.740741,0.318182,0.124518,0.0,0.166667,0.889126,0.294118,0.174889,0.166603,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,0,0.00831
4,1,5,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,1.0,0.668277,0.242424,0.149960,0.0,0.255952,0.746269,0.235294,0.174734,0.402078,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,0,0.01108


In [ ]:
test_df['cycle_norm'] = test_df['cycle']
norm_test_df = pd.DataFrame(min_max_scaler.transform(test_df[cols_normalize]), 
                            columns=cols_normalize, 
                            index=test_df.index)
test_join_df = test_df[test_df.columns.difference(cols_normalize)].join(norm_test_df)
test_df = test_join_df.reindex(columns = test_df.columns)
test_df = test_df.reset_index(drop=True)
test_df.head()


,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,cycle_norm
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,1.0,0.652174,0.212121,0.127614,0.0,0.208333,0.646055,0.220588,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.00000
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,1.0,0.805153,0.166667,0.146684,0.0,0.386905,0.739872,0.264706,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.00277
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,1.0,0.685990,0.227273,0.158081,0.0,0.386905,0.699360,0.220588,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.00554
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,1.0,0.679549,0.196970,0.105717,0.0,0.255952,0.573561,0.250000,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.00831
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,1.0,0.694042,0.166667,0.102396,0.0,0.273810,0.737740,0.220588,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.01108


In [ ]:
test_df

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,cycle_norm
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,1.0,0.652174,0.212121,0.127614,0.0,0.208333,0.646055,0.220588,0.132160,0.308965,0.0,0.333333,0.0,0.0,0.558140,0.661834,0.000000
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,1.0,0.805153,0.166667,0.146684,0.0,0.386905,0.739872,0.264706,0.204768,0.213159,0.0,0.416667,0.0,0.0,0.682171,0.686827,0.002770
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,1.0,0.685990,0.227273,0.158081,0.0,0.386905,0.699360,0.220588,0.155640,0.458638,0.0,0.416667,0.0,0.0,0.728682,0.721348,0.005540
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,1.0,0.679549,0.196970,0.105717,0.0,0.255952,0.573561,0.250000,0.170090,0.257022,0.0,0.250000,0.0,0.0,0.666667,0.662110,0.008310
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,1.0,0.694042,0.166667,0.102396,0.0,0.273810,0.737740,0.220588,0.152751,0.300885,0.0,0.166667,0.0,0.0,0.658915,0.716377,0.011080
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,100,194,0.781609,0.500000,0.0,0.0,0.611446,0.619359,0.566172,0.0,1.0,0.573269,0.181818,0.541326,0.0,0.500000,0.426439,0.176471,0.584890,0.564063,0.0,0.500000,0.0,0.0,0.395349,0.418669,0.534626
13092,100,195,0.436782,0.416667,0.0,0.0,0.605422,0.537388,0.671843,0.0,1.0,0.542673,0.227273,0.533743,0.0,0.446429,0.503198,0.308824,0.572350,0.485956,0.0,0.583333,0.0,0.0,0.333333,0.528721,0.537396
13093,100,196,0.465517,0.250000,0.0,0.0,0.671687,0.482014,0.414754,0.0,1.0,0.513688,0.318182,0.561249,0.0,0.428571,0.530917,0.235294,0.605326,0.507888,0.0,0.583333,0.0,0.0,0.372093,0.429301,0.540166
13094,100,197,0.281609,0.583333,0.0,0.0,0.617470,0.522128,0.626435,0.0,1.0,0.566828,0.257576,0.570403,0.0,0.452381,0.562900,0.294118,0.622046,0.562524,0.0,0.583333,0.0,0.0,0.403101,0.518779,0.542936


In [ ]:
# Split dataset into 2 subdatasets

split = 50

sum1 = split
sum2 = 100 - split


train_df_1 = train_df[ train_df.id <= split ]
train_df_2 = train_df[ (train_df.id > split) & (train_df.id <= 100)  ]



print(f"""Split dataset into 3
Dataset 1:   {sum1} engines and {len(train_df_1.index)} cycles, Average: {len(train_df_1.index)/sum1} cycles
Dataset 2:   {sum2} engines and {len(train_df_2.index)} cycles, Average: {len(train_df_2.index)/sum2} cycles
""")

Split dataset into 3
Dataset 1:   50 engines and 9909 cycles, Average: 198.18 cycles
Dataset 2:   50 engines and 10722 cycles, Average: 214.44 cycles



In [ ]:
# Split dataset into 3 subdatasets

# split1 = 20
# split2 = 60

# sum1 = split1
# sum2 = split2 - split1
# sum3 = 100 - split2


# train_df_1 = train_df[ train_df.id <= split1 ]
# train_df_2 = train_df[ (train_df.id >split1) & (train_df.id <= split2)  ]
# train_df_3 = train_df[ (train_df.id > split2) & (train_df.id <= 100)  ]



# print(f"""Split dataset into 3
# Dataset 1:   {sum1} engines and {len(train_df_1.index)} cycles, Average: {len(train_df_1.index)/sum1} cycles
# Dataset 2:   {sum2} engines and {len(train_df_2.index)} cycles, Average: {len(train_df_2.index)/sum2} cycles
# Dataset 3:   {sum3} engines and {len(train_df_3.index)} cycles, Average: {len(train_df_3.index)/sum3} cycles
# """)

In [ ]:
#
# TEST DATA
#

In [ ]:
# generate column max for test data
rul = pd.DataFrame(test_df.groupby('id')['cycle'].max()).reset_index()
rul.columns = ['id', 'max']
truth_df.columns = ['more']
truth_df['id'] = truth_df.index + 1
truth_df['max'] = rul['max'] + truth_df['more']
truth_df.drop('more', axis=1, inplace=True)

In [ ]:
# generate RUL for test data
test_df = test_df.merge(truth_df, on=['id'], how='left')
test_df['RUL'] = test_df['max'] - test_df['cycle']
test_df.drop('max', axis=1, inplace=True)
# test_df.head()



In [ ]:
# generate label columns w0 and w1 for test data
test_df['fail_30'] = np.where(test_df['RUL'] <= w1, 1, 0 )
# test_df['fail_15'] = test_df['fail_30']
# test_df.loc[test_df['RUL'] <= w0, 'fail_15'] = 2
# test_df.head()

In [ ]:
# test_df_1 = test_df[ test_df.id <= split1 ]
# test_df_2 = test_df[ (test_df.id >split1) & (test_df.id <= split2)  ]
# test_df_3 = test_df[ (test_df.id > split2) & (test_df.id <= 100)  ]

In [ ]:
test_df[test_df["fail_30"] == 1]

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL,fail_30
2113,18,131,-0.0035,-0.0002,100.0,518.67,643.29,1594.24,1408.49,14.62,21.61,554.06,2388.13,9065.96,1.3,47.49,521.33,2388.13,8133.06,8.4389,0.03,394,2388,100.0,38.74,23.1609,30,1
2114,18,132,0.0016,0.0004,100.0,518.67,642.85,1599.11,1413.85,14.62,21.61,552.70,2388.19,9048.47,1.3,47.74,521.27,2388.10,8133.28,8.4285,0.03,393,2388,100.0,38.63,23.2268,29,1
2115,18,133,0.0007,0.0001,100.0,518.67,643.00,1600.45,1419.18,14.62,21.61,553.36,2388.14,9056.09,1.3,47.70,521.06,2388.19,8136.80,8.4418,0.03,395,2388,100.0,38.71,23.2637,28,1
2420,20,170,0.0011,0.0003,100.0,518.67,643.16,1599.33,1418.72,14.62,21.61,552.55,2388.09,9071.92,1.3,47.82,520.54,2388.14,8144.74,8.5027,0.03,394,2388,100.0,38.71,23.2006,30,1
2421,20,171,0.0013,-0.0005,100.0,518.67,642.95,1594.47,1425.58,14.62,21.61,552.90,2388.15,9069.08,1.3,47.87,520.56,2388.13,8143.27,8.4611,0.03,394,2388,100.0,38.68,23.2134,29,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,100,194,0.0049,0.0000,100.0,518.67,643.24,1599.45,1415.79,14.62,21.61,553.41,2388.02,9142.37,1.3,47.69,520.69,2388.00,8213.28,8.4715,0.03,394,2388,100.0,38.65,23.1974,24,1
13092,100,195,-0.0011,-0.0001,100.0,518.67,643.22,1595.69,1422.05,14.62,21.61,553.22,2388.05,9140.68,1.3,47.60,521.05,2388.09,8210.85,8.4512,0.03,395,2388,100.0,38.57,23.2771,23,1
13093,100,196,-0.0006,-0.0003,100.0,518.67,643.44,1593.15,1406.82,14.62,21.61,553.04,2388.11,9146.81,1.3,47.57,521.18,2388.04,8217.24,8.4569,0.03,395,2388,100.0,38.62,23.2051,22,1
13094,100,197,-0.0038,0.0001,100.0,518.67,643.26,1594.99,1419.36,14.62,21.61,553.37,2388.07,9148.85,1.3,47.61,521.33,2388.08,8220.48,8.4711,0.03,395,2388,100.0,38.66,23.2699,21,1


In [ ]:
# Save datasets

train_df.to_csv( path +  "TRAINING_SET_FULL.csv", index=False)

train_df_1.to_csv( path +  "TRAINING_SET_1.csv", index=False)
train_df_2.to_csv( path +  "TRAINING_SET_2.csv", index=False)
# train_df_3.to_csv( path +  "TRAINING_SET_3.csv", index=False)

# test_df_1.to_csv( path +  "TEST_SET_1.csv", index=False)
# test_df_2.to_csv( path +  "TEST_SET_2.csv", index=False)
# test_df_3.to_csv( path +  "TEST_SET_3.csv", index=False)

test_df.to_csv( path +  "TEST_SET_FULL.csv", index=False)


In [ ]:
# # Save datasets non normalized

# path += "final_datasets_non_normalized/"

# train_df.to_csv( path +  "TRAINING_SET_FULL.csv", index=False)

# train_df_1.to_csv( path +  "TRAINING_SET_1.csv", index=False)
# train_df_2.to_csv( path +  "TRAINING_SET_2.csv", index=False)
# # train_df_3.to_csv( path +  "TRAINING_SET_3.csv", index=False)

# # test_df_1.to_csv( path +  "TEST_SET_1.csv", index=False)
# # test_df_2.to_csv( path +  "TEST_SET_2.csv", index=False)
# # test_df_3.to_csv( path +  "TEST_SET_3.csv", index=False)

# test_df.to_csv( path +  "TEST_SET_FULL.csv", index=False)

In [ ]:
# Save test engines as separated csv files

ids = pd.unique(test_df.id)
engines = []
for id in ids:
  engines.append(test_df[ test_df.id == id ])
  engines[id-1].to_csv( path + f"test_engines_normalized_seperated/test_engine_{id}.csv", index=False)
  # print(id)
test_engine = engines[34]


Classification Dataset

In [1]:
train_df

NameError: ignored